## Importing Libraries

In [1]:
import numpy as np
import pandas as pd
from glob import glob
import xml.etree.ElementTree as xet
import cv2
import os
from shutil import copy

## Extract xmax, xmin, ymax, ymin of the bounding box of the image and save them in a csv file.

In [2]:
path = glob('dataset/annotations/*.xml')
labels_dict = dict(filepath=[],xmin=[],xmax=[],ymin=[],ymax=[])
for filename in path:

    info = xet.parse(filename)
    root = info.getroot()
    member_object = root.find('object')
    labels_info = member_object.find('bndbox')
    xmin = int(labels_info.find('xmin').text)
    xmax = int(labels_info.find('xmax').text)
    ymin = int(labels_info.find('ymin').text)
    ymax = int(labels_info.find('ymax').text)

    labels_dict['filepath'].append(filename)
    labels_dict['xmin'].append(xmin)
    labels_dict['xmax'].append(xmax)
    labels_dict['ymin'].append(ymin)
    labels_dict['ymax'].append(ymax)

In [3]:
df = pd.DataFrame(labels_dict)
df.to_csv('dataset/labels.csv',index=False)
df.head()

filepath  xmin  xmax  ymin  ymax
0    dataset/annotations\Cars0.xml   226   419   125   173
1    dataset/annotations\Cars1.xml   134   262   128   160
2   dataset/annotations\Cars10.xml   140   303     5   148
3  dataset/annotations\Cars100.xml   175   214   114   131
4  dataset/annotations\Cars101.xml   167   240   202   220

In [4]:
df = pd.read_csv("dataset/labels.csv")
df.head()

filepath  xmin  xmax  ymin  ymax
0    dataset/annotations\Cars0.xml   226   419   125   173
1    dataset/annotations\Cars1.xml   134   262   128   160
2   dataset/annotations\Cars10.xml   140   303     5   148
3  dataset/annotations\Cars100.xml   175   214   114   131
4  dataset/annotations\Cars101.xml   167   240   202   220

 ## As we need x_center, y_center, width and height of our bounding box, so lets calculate them.

### 1) Calculating width, height of input image

In [5]:
def parsing(path):
    parser = xet.parse(path).getroot()
    name = parser.find("filename").text
    filename = f"dataset/images/{name}"

    # width and height
    parser_size = parser.find("size")
    width = int(parser_size.find("width").text)
    height = int(parser_size.find("height").text)
    
    return filename, width, height

In [6]:
df[["image_filename", "width", "height"]] = df["filepath"].apply(parsing).apply(pd.Series)

df.head()

filepath  xmin  xmax  ymin  ymax  \
0    dataset/annotations\Cars0.xml   226   419   125   173   
1    dataset/annotations\Cars1.xml   134   262   128   160   
2   dataset/annotations\Cars10.xml   140   303     5   148   
3  dataset/annotations\Cars100.xml   175   214   114   131   
4  dataset/annotations\Cars101.xml   167   240   202   220   

               image_filename  width  height  
0    dataset/images/Cars0.png    500     268  
1    dataset/images/Cars1.png    400     248  
2   dataset/images/Cars10.png    400     225  
3  dataset/images/Cars100.png    400     267  
4  dataset/images/Cars101.png    400     300

### 2) Calculating x_center, y_center, width and height of the bounding-box(bb)

In [7]:
df["x_center"] = (df["xmax"] + df["xmin"]) / (2 * df["width"])
df["y_center"] = (df["ymax"] + df["ymin"]) / (2 * df["height"])

df["bb_width"] = (df["xmax"] - df["xmin"]) / df["width"]
df["bb_height"] = (df["ymax"] - df["ymin"]) / df["height"]

df

filepath  xmin  xmax  ymin  ymax  \
0      dataset/annotations\Cars0.xml   226   419   125   173   
1      dataset/annotations\Cars1.xml   134   262   128   160   
2     dataset/annotations\Cars10.xml   140   303     5   148   
3    dataset/annotations\Cars100.xml   175   214   114   131   
4    dataset/annotations\Cars101.xml   167   240   202   220   
..                               ...   ...   ...   ...   ...   
428   dataset/annotations\Cars95.xml   278   407   182   263   
429   dataset/annotations\Cars96.xml   133   261   126   160   
430   dataset/annotations\Cars97.xml    98   297   102   191   
431   dataset/annotations\Cars98.xml    85   247   196   259   
432   dataset/annotations\Cars99.xml    70   262   198   271   

                 image_filename  width  height  x_center  y_center  bb_width  \
0      dataset/images/Cars0.png    500     268  0.645000  0.555970  0.386000   
1      dataset/images/Cars1.png    400     248  0.495000  0.580645  0.320000   
2     dataset/images/Cars10.png    400     225  0.553750  0.340000  0.407500   
3    dataset/images/Cars100.png    400     267  0.486250  0.458801  0.097500   
4    dataset/images/Cars101.png    400     300  0.508750  0.703333  0.182500   
..                          ...    ...     ...       ...       ...       ...   
428   dataset/images/Cars95.png    600     400  0.570833  0.556250  0.215000   
429   dataset/images/Cars96.png    400     248  0.492500  0.576613  0.320000   
430   dataset/images/Cars97.png    400     300  0.493750  0.488333  0.497500   
431   dataset/images/Cars98.png    370     400  0.448649  0.568750  0.437838   
432   dataset/images/Cars99.png    375     500  0.442667  0.469000  0.512000   

     bb_height  
0     0.179104  
1     0.129032  
2     0.635556  
3     0.063670  
4     0.060000  
..         ...  
428   0.202500  
429   0.137097  
430   0.296667  
431   0.157500  
432   0.146000  

[433 rows x 12 columns]

## Splitting the data to Train-Test Split and storing the data in seperate folders

In [8]:
df_train = df.iloc[:320]
df_test = df.iloc[320:]

In [10]:
train_folder = "data_images/train"

values = df_train[["image_filename", "x_center", "y_center", "bb_width", "bb_height"]].values
for fname,x,y,w,h in values:
    image_name = os.path.split(fname)[-1]
    txt_name = os.path.splitext(image_name)[0]
    
    dst_image_path = os.path.join(train_folder, image_name)
    dst_label_file = os.path.join(train_folder, txt_name+".txt")
    
    copy(fname, dst_image_path)

    label_txt = f"0 {x} {y} {w} {h}"
    with open(dst_label_file, mode="w") as f:
        f.write(label_txt)
        f.close()

In [11]:
test_folder = "data_images/test"

values = df_test[["image_filename", "x_center", "y_center", "bb_width", "bb_height"]].values
for fname,x,y,w,h in values:
    image_name = os.path.split(fname)[-1] 
    txt_name = os.path.splitext(image_name)[0] 
    
    dst_image_path = os.path.join(test_folder, image_name) 
    dst_label_file = os.path.join(test_folder, txt_name+".txt")

    copy(fname, dst_image_path)

    label_txt = f"0 {x} {y} {w} {h}"
    with open(dst_label_file, mode="w") as f:
        f.write(label_txt)
        f.close()